In [1]:
import pandas as pd
import numpy as np
import pymysql

In [2]:
import os

In [3]:
os.listdir('./')

['.ipynb_checkpoints',
 'dalla_data_labeling.xlsx',
 'dalla_raw_data.csv',
 'dalla_test.xlsx',
 'dalla_train_data.csv',
 'dalla_vaildation.csv',
 'dalla_validation_label.xlsx',
 'fanboard_data.ipynb',
 'NavieModel38.pkl',
 'NavieVectorizer38.pkl',
 'navie_train.csv',
 'navie_train.ipynb',
 'random_sampling.csv']

In [4]:
dalla_data = pd.read_csv('dalla_train_data.csv')
dalla_data

,contents,label
0,일이 있으면 공지해준다매요...공지도 없구 잠수야?,0
1,응 2023년도 1월1일 첫팬보드 내꺼? 새해복 많이받아라:),0
2,:?,0
3,여기 팬보드는 나의것..!,0
4,팬보드 1빠! 발도장 꾹!?,0
...,...,...
1394,물은 삼다수!,0
1395,아지야 미안해 ㅋㅋ 댓글 봤는데 내 손이 더 빨랐어 흐흐흫 담에 봐???,0
1396,"너무 목소리 좋아요 , 눈나아~",0
1397,안녕하세요 ??‍♀️다름이 아니라 1월15일옹기종기 1주년축하메세지롤링페이퍼를 작성...,0


In [5]:
dalla_data['label'].value_counts()

0    1090
1     309
Name: label, dtype: int64

----

# Navie bayes model Trainning

In [6]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [7]:
dalla_data = dalla_data.dropna()
dalla_data.isnull().sum()

contents    0
label       0
dtype: int64

#### 기존에 만들어뒀던 vectorizer가 3.8에서 pickle로 만들어져서, 3.6에서 호환이 안되서 dataframe으로 불러와서 다시 만들기

In [8]:
total_df = pd.read_csv('navie_train.csv')
total_df

,contents,label
0,엄마 그거아니야 ........이렇게 됄까봐 말안하고 갈러고 한건대내가잘못했어.....,0
1,입장 /:) 자기소개하고 우리 친해질까요 ?달/반짝반짝 빛나는 너의 매력에 퐁당 !...,0
2,[멜론TOP 2023 활동평가상 공지]팀 활동평가상(Team Activity Eva...,0
3,희야! 앞으로 방송에 자주 놀러 갈게! 이게 라는구나!ㅎㅎ 오늘 나 한테 모르는 ...,0
4,왕 딸기였는데 맛있더라 ?,0
...,...,...
813,⌛️? 급하다 급해! 영입 마감까지 남은 시간이 별로 없어요! ?⌛️.Spoon R...,1
814,⌛️? 급하다 급해! 영입 마감까지 남은 시간이 별로 없어요! ?⌛️.Spoon R...,1
815,.Spoon Radio DJ 사업 담당자입니다.갑작스러운 에 놀라셨겠지만 부담없이 ...,1
816,⌛️? 급하다 급해! 영입 마감까지 남은 시간이 별로 없어요! ?⌛️.Spoon R...,1


In [9]:
total_df['contents'] = total_df['contents'].str.replace('팬보드','').str.replace('안녕하세요','').str.replace('감사합니다','')

In [10]:
# contervectorizer

a_data = total_df['contents'].values.astype('U')
a_data

array(['엄마 그거아니야 ........이렇게 됄까봐 말안하고 갈러고 한건대내가잘못했어.....',
       '입장 /:) 자기소개하고 우리 친해질까요 ?달/반짝반짝 빛나는 너의 매력에 퐁당 !!감사/τнänκ чöü♥ 항상 고맙습니다!하트/하트  !?방종전/ 즐거웠어요? 그럼 이만 총총•*¨*•.¸¸ઇଓ',
       '[멜론TOP 2023 활동평가상 공지]팀 활동평가상(Team Activity Evaluation Award) T.A.E.A1월31일 우리팀 자체 팀포인트 합산 기준으로 순위별로 팀활동평가상 진행-평가기준팀내순위- BJ , 시청자 /팀포인트/ 팀내 방송참여 와 시청시간(입컷, 30분이내 시청은 제외)팀외부- 시청자중 우리팀 방송 참여 횟수및 듣방시간 기준(입컷, 30분이내 시청제외)*순위 별로 1등 100달 2등 50달 3등 장려상 50달 포상Ps. 많은 팀참여 및 활동 바랍니다',
       '희야! 앞으로 방송에 자주 놀러 갈게! 이게  라는구나!ㅎㅎ 오늘 나 한테 모르는 거 알려줘서 정말 고마워 내가 라이브가 진짜! 처음이여서 정말 낯설었는데 나를 신경을 좀 써줘서 고마워! 사랑해! 애정하고!ㅎㅎ ><? ',
       '왕 딸기였는데 맛있더라 ?', '좋아요 : 191',
       '엄마 그거 알아 ?나도 엄마한태 너무 집착해서다른사람들 만나려고 했는대다 이용만하고버리거나 무서워서 못다가갔더라.....',
       '은수야 오늘 즐거웠당 ㅎㅎ',
       '아~~ 다피님이었구나일부러 사람들 보라고 이렇게적는건아니니까 오해하지마 ㅋㄱ그치 좋아하거나 사랑할수있으니깐계속 기다려봐',
       '저어 그 밍 구넴 저 여운인데요 음 기다리게 한건 사과안받아도대구 일단 행동이 이해가서요 :) 새해복 많이 받으시구 음..이런 경험 처음이라서 저기 너무 걱정 마시구 그냥 니가 너무 부담스럽게 한 죄야 라고 하고 지내심대여 헤헿.. 괜찮슴다 저도 이렇게 애같아보기 진짜 ㅋㅋㅋ 첨이엇어서 ㅜㅜ 송생님 고마워요 슬프긴한데 괜찮슴

In [11]:
vectorizer = CountVectorizer()
vectorizer.fit(a_data)

CountVectorizer()

In [12]:
vectorizer.vocabulary_

{'엄마': 2279,
 '그거아니야': 528,
 '이렇게': 2546,
 '됄까봐': 971,
 '말안하고': 1213,
 '갈러고': 354,
 '한건대내가잘못했어': 3394,
 '입장': 2649,
 '자기소개하고': 2716,
 '우리': 2461,
 '친해질까요': 3159,
 '반짝반짝': 1455,
 '빛나는': 1709,
 '너의': 766,
 '매력에': 1243,
 '퐁당': 3287,
 '감사': 358,
 'τнänκ': 235,
 'чöü': 236,
 '항상': 3434,
 '고맙습니다': 451,
 '하트': 3390,
 '방종전': 1515,
 '즐거웠어요': 2964,
 '그럼': 552,
 '이만': 2554,
 '총총': 3120,
 'ઇଓ': 237,
 '멜론top': 1278,
 '2023': 67,
 '활동평가상': 3576,
 '공지': 477,
 'team': 228,
 'activity': 136,
 'evaluation': 166,
 'award': 140,
 'a1월31일': 132,
 '우리팀': 2466,
 '자체': 2733,
 '팀포인트': 3251,
 '합산': 3433,
 '기준으로': 606,
 '순위별로': 1946,
 '팀활동평가상': 3255,
 '진행': 3019,
 '평가기준팀내순위': 3281,
 'bj': 145,
 '시청자': 1999,
 '팀내': 3230,
 '방송참여': 1499,
 '시청시간': 1995,
 '입컷': 2652,
 '30분이내': 92,
 '시청은': 1996,
 '제외': 2858,
 '팀외부': 3237,
 '시청자중': 2001,
 '방송': 1479,
 '참여': 3055,
 '횟수및': 3588,
 '듣방시간': 1025,
 '기준': 604,
 '시청제외': 2002,
 '순위': 1944,
 '별로': 1540,
 '1등': 49,
 '100달': 13,
 '2등': 81,
 '50달': 105,
 '3등': 94,
 '장려상': 2766,
 '포

In [13]:
counts = vectorizer.transform(a_data)
print(counts.shape)

(818, 3631)


In [14]:
multi_classification = MultinomialNB()

labels = [0] * 509 + [1] *309

multi_classification.fit(counts, labels)

MultinomialNB()

In [15]:
print(multi_classification.predict(counts))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [16]:
print(multi_classification.predict_proba(counts))

[[9.99998273e-001 1.72715737e-006]
 [1.00000000e+000 1.31248342e-028]
 [1.00000000e+000 1.51284682e-075]
 ...
 [2.92941751e-187 1.00000000e+000]
 [1.83320350e-231 1.00000000e+000]
 [2.35409258e-188 1.00000000e+000]]


In [17]:
case1 = "안녕하세요.Spoon Radio DJ 사업 담당자입니다.갑작스러운 팬보드에 놀라셨겠지만 부담없이 아래 내용을 살펴보시면 좋겠어요!우선 Spoon Radio는 누구나 목소리 하나로 자유롭게 소통하면서 후원 아이템을 통해 수익까지 창출할 수 있는 오디오 플랫폼입니다.국내 최대 규모의 오디오 플랫폼으로 한발 더 나아가기 위해 Spoon Radio는 보다 다양한 매력을 가진 DJ를 찾고 있는데요.해나잉뎅님이 이러한 저희의 니즈에 부합하는 크리에이터라고 판단되어 이렇게 팬보드를 남기게 되었습니다.혹여 달라 안에서의 대화에 불편함이 있을 해나잉뎅님을 위해Spoon Radio 공식 계정(Spoon FM)에 팬보드를 '나만보기'로 아래 내용을 남겨주시면 지원 내용과 혜택 등을 보다 자세히 안내드리도록 하겠습니다.① Spoon FM url : https://www.spooncast.net/kr/profile/6343043/board/dj?t=all② 남기실 내용 : 달라 계정 정보(닉네임, 프로필 url)와 이메일 주소신규 DJ를 위한 이벤트도 준비하고 있으니 많은 관심 부탁드려요!감사합니다."
print(multi_classification.predict(vectorizer.transform([case1])))
print(multi_classification.predict_proba(vectorizer.transform([case1])))

[1]
[[8.73956871e-186 1.00000000e+000]]


In [18]:
case2 = "누나 안녕? 이번주에 누나 코로나로... 많이 힘들었을텐데... 잘 이겨낸 것같아 다행인거 같아~ 편지는 적고 싶은데 어디에 적을까 고민하다가 이번엔 이렇게 팬보드에다가 적게 되었어~ 앞으로 안아픈게 젤 좋지만 만약 또 이런 아픔이 생긴다면, 나를 포함한(?)고정팬들이 누나를 걱정하고 있다는 걸 생각하면서 버텨내길 바래~~ 오늘도 방송하느라 수고 많았고, 목 다시 안좋아졌는데 푹 쉬면서 빠른 시일내로 원래대로 몸상태가 돌아와서 예전처럼 열심히 방송하는 민이 누나가 되길 바래~~ 어제 장난 좀 많이 친거 가튼데.... 미안해애ㅠㅠ 티는 안내고 있지만 누나 많이 걱정하고 있어~ 알바중에도 누나 생각나면 편지도 적어보고 할 정도루~~ 힘들 때마다 누나 생각하면서 버티는중이야~~ 어제였나? 사연으로 올해 연말까지 10~20등 사이에 들어가겠다고 한거 최소 30등까지 가볼려고 생각중이야... 헷(?) 오늘 누나랑 달라에서 본지 한달 조금 넘은 시점인데 한달 동안 너무 누나로 인해 행복하게 지낸것 같아... 앞으로는 아프지 말고 건강하게 방송도 하고 누나 일상생활도 잘 하길 바래~~ 너무 자고만 있지 말고, 산책도 좀 하면서~~ 오늘 편지는 여기까지 할게~~ 고맙고 많이 애정해"
print(multi_classification.predict(vectorizer.transform([case2])))
print(multi_classification.predict_proba(vectorizer.transform([case2])))

[0]
[[1.00000000e+00 7.99820971e-72]]


In [19]:
case3 = "스푼을 들어서 밥을 먹는다"

print(multi_classification.predict(vectorizer.transform([case3])))
print(multi_classification.predict_proba(vectorizer.transform([case3])))

[0]
[[0.99093246 0.00906754]]


In [20]:
case4 = "spoon"

print(multi_classification.predict(vectorizer.transform([case4])))
print(multi_classification.predict_proba(vectorizer.transform([case4])))

[1]
[[0.00620465 0.99379535]]


In [21]:
case5 = "아지야 미안해 ㅋㅋ 댓글 봤는데 내 손이 더 빨랐어 \n흐흐흫 담에 봐???"

print(multi_classification.predict(vectorizer.transform([case5])))
print(multi_classification.predict_proba(vectorizer.transform([case5])))

[0]
[[9.99988506e-01 1.14938170e-05]]


In [22]:
case6 = "spoon, fork, nife, 스푼에서 활동할 생각 없으세요?"

print(multi_classification.predict(vectorizer.transform([case6])))
print(multi_classification.predict_proba(vectorizer.transform([case6])))

[1]
[[0.45307682 0.54692318]]


----
# 2023 데이터로 navie 확인

In [31]:
import pymysql

In [35]:
df = data[['contents']]
df

,contents
0,:?
1,응 2023년도 1월1일 첫팬보드 내꺼? 새해복 많이받아라:)
2,2023년도두 파이팅ㅎㅎ
3,⭐️
4,응응 우리 솜이도
...,...
42157,S-코우지
42158,우리 길묘 항상 힘내쟈???
42159,대박 팀 저거 너무 ｡ﾟﾟ･｡･ﾟﾟ｡ ﾟ。 귀 ｡ﾟ ﾟ･｡･ﾟ ｡ ﾟﾟ･｡･...
42160,아라누나 lintime3 갠톡 하나만 보내주면 상황설명 해줄께


In [26]:
from tqdm import tqdm
tqdm.pandas() 


In [36]:
df['predict'] = df['contents'].progress_apply(lambda x: int(''.join(map(str,multi_classification.predict(vectorizer.transform([x]))))))

100%|██████████████████████████████████████████████████████████████████████████| 42162/42162 [00:06<00:00, 6578.12it/s]
C:\Users\PC\.conda\envs\py36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [38]:
df['predict_proba'] =  df['contents'].progress_apply(lambda x: round(multi_classification.predict_proba(vectorizer.transform([x])[0])[0][1]*100,2))

100%|██████████████████████████████████████████████████████████████████████████| 42162/42162 [00:12<00:00, 3463.39it/s]
C:\Users\PC\.conda\envs\py36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [39]:
df

,contents,predict,predict_proba
0,:?,0,37.78
1,응 2023년도 1월1일 첫팬보드 내꺼? 새해복 많이받아라:),0,0.00
2,2023년도두 파이팅ㅎㅎ,0,37.78
3,⭐️,0,37.78
4,응응 우리 솜이도,0,0.87
...,...,...,...
42157,S-코우지,0,37.78
42158,우리 길묘 항상 힘내쟈???,0,0.00
42159,대박 팀 저거 너무 ｡ﾟﾟ･｡･ﾟﾟ｡ ﾟ。 귀 ｡ﾟ ﾟ･｡･ﾟ ｡ ﾟﾟ･｡･...,0,0.04
42160,아라누나 lintime3 갠톡 하나만 보내주면 상황설명 해줄께,0,0.91


In [40]:
for idx, row in df.loc[df['predict']==1].iterrows():
    print(f" Naive 기반 확률 -> {row['predict_proba']}")
    print(f"{row['contents']}")

 Naive 기반 확률 -> 96.97
이상 끝
 Naive 기반 확률 -> 68.26
22년에 무르옵이라는 좋은 인연을 알게 돼서 쪼와?
 Naive 기반 확률 -> 66.86
넌 나 옆에 둔거 후회안해? 3년만에 이렇게 변해버렸는데 쩝..뭐 아무튼
 Naive 기반 확률 -> 97.44
피곤해서 휴방했죠 아침방 있으니 오세요
 Naive 기반 확률 -> 84.41
23년도 잘 부탁드려요?‍♂️
 Naive 기반 확률 -> 73.94
츄즈언니를 통해 친해져서 넘 좋숩니당
 Naive 기반 확률 -> 68.26
좋은 사람에게는 복이 가득해야지 ???
 Naive 기반 확률 -> 55.14
또 한해가 지났구나~\n참 시간이 빠르다...\n우리는 언제 보냐...
 Naive 기반 확률 -> 66.07
2023년에도\n행복하자\n넌 최고의 여캠인걸\n잊지마 \n싀발 달라 대상이 인정한\n1터어임 ㅋㅋㅋㅋㅋㅋ
 Naive 기반 확률 -> 84.41
벼리누나두 새복많받! ?\n올해에두 잘 부탁드려요 ???\n
 Naive 기반 확률 -> 66.07
후욱후욱.... 달라 어워즈 대상.... 오뉴오빠가아..... 팬보드 남겨주움....???
 Naive 기반 확률 -> 66.86
아뉘... 휴방날이면서 킬것처럼 설레게 해놓고 이렇게 안와버리면 나 잠 못잔다... 책임져 ... 보고싶쟈낭...? 씌익씌익
 Naive 기반 확률 -> 99.93
서울시가 지하철, 시내버스, 마을버스 요금 인상을 추진한다고 29일 밝혔다. 지난 2015년 6월 지하철 200원, 버스 기본요금 150원 인상한 이후 만 7년 6개월 만의 결정이다. 요금은 현행보다 300원 더 오를 것으로 보인다. \n\n서울시는 대중교통 요금 조정을 위해 연내 경기도·인천시 등 통합환승할인제에 참여하고 있는 관계 기관과 협의를 시작하고, 시민 공청회, 요금조정계획에 대한 시의회 의견청취, 물가대책심의위원회 심의 등을 거쳐 이르면 4월 말 요금을 조정할 계획이다.\n\n시는 \교통복지로서 민생 경제에 직접적인 영향을

In [41]:
for idx, row in df.loc[df['predict_proba']>=99].iterrows():
    print(row['contents'])

서울시가 지하철, 시내버스, 마을버스 요금 인상을 추진한다고 29일 밝혔다. 지난 2015년 6월 지하철 200원, 버스 기본요금 150원 인상한 이후 만 7년 6개월 만의 결정이다. 요금은 현행보다 300원 더 오를 것으로 보인다. \n\n서울시는 대중교통 요금 조정을 위해 연내 경기도·인천시 등 통합환승할인제에 참여하고 있는 관계 기관과 협의를 시작하고, 시민 공청회, 요금조정계획에 대한 시의회 의견청취, 물가대책심의위원회 심의 등을 거쳐 이르면 4월 말 요금을 조정할 계획이다.\n\n시는 \교통복지로서 민생 경제에 직접적인 영향을 줄 수 있는 대중교통 요금 인상을 최대한 늦춰왔으나, 정부 예산안에서 지방자치단체의 지하철 무임손실 지원 예산이 제외되면서 운영 어려움이 더욱 심화된 만큼 부득이하게 요금을 인상할 수밖에 없는 상황\이라고 시민들의 양해를 구했다.
Spoon
안녕하세요. \n\n스푼라디오 DJ 사업 담당자입니다. \n\n갑작스러운 팬보드에 놀라셨겠지만 부담없이 아래 내용을 살펴보시면 좋겠습니다. \n\n우선 스푼라디오는 누구나 목소리 하나로 자유롭게 소통하면서 ‘스푼’이라는 아이템을 통해 수익까지 창출할 수 있는 오디오 플랫폼입니다. \n\n국내 최대 규모의 오디오 플랫폼으로 한발 더 나아가기 위해 스푼라디오는 보다 다양한 매력을 가진 DJ를 찾고 있는데요. \n\n** 님이 이러한 저희의 니즈에 부합하는 크리에이터라고 판단되어 이렇게 팬보드를 남기게 되었습니다. \n\n혹여 달라 안에서의 대화에 불편함이 있을 것으로 판단되어, \n\n스푼라디오 공식 계정(스푼FM)에 팬보드를 남겨주시면 지원 내용과 혜택 등을 보다 자세히 안내드리도록 하겠습니다. \n\nhttps://www.spooncast.net/kr/profile/6343043/board/dj?t=all\n\n신규 DJ를 위한 이벤트도 준비하고 있으니 많은 관심 부탁드려요!\n감사합니다.
Spoon
Spoon Radio
안녕하세요. \n\nSpoon Radio DJ 사업 담당자입니다. \n

    - 불용어로 안녕하세요, 감사합니다, 팬보드 삭제하기
    - 확률 99% 이상만 거르기 하면 나이브 베이즈로 거를 수 있음


## Navie Bayes Classification 모델, Vectorizer 저장

In [42]:
import joblib
import pickle

In [43]:
joblib.dump(multi_classification, 'NavieModel.pkl')

['NavieModel.pkl']

In [44]:
model = joblib.load('NavieModel.pkl')
model

MultinomialNB()

In [ ]:
vectorizer

In [45]:
joblib.dump(vectorizer, 'navie_vectorizer.pkl')

['navie_vectorizer.pkl']

In [54]:
NaiveVectorizer = joblib.load('navieVectorizer.pkl')
NaiveVectorizer

CountVectorizer()

### load 후 테스트

In [55]:
print(model.predict(NaiveVectorizer.transform(['spoon Radio 공식 계정(Spoon FM)에 팬보드를 남겨주시면 지원 내용과 혜택 등을 보다 자세히 안내드리도록 하겠습니다.'])))
print(model.predict_proba(NaiveVectorizer.transform(['spoon Radio 공식 계정(Spoon FM)에 팬보드를 남겨주시면 지원 내용과 혜택 등을 보다 자세히 안내드리도록 하겠습니다.'])))

[1]
[[6.93746068e-28 1.00000000e+00]]


In [52]:
int(''.join(map(str,model.predict(NaiveVectorizer.transform(['spoon'])))))

1

In [50]:
round(model.predict_proba(NaiveVectorizer.transform(['테스트']))[0][1]*100,2)

37.78